# StackTheory LLM/ML Demo 1


In [1]:
#!pip install chroma-haystack

In [2]:
import torch
from haystack import Pipeline, Document
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.writers import DocumentWriter
# Create pipeline components
from haystack_integrations.components.retrievers.chroma import ChromaQueryTextRetriever
from haystack_integrations.document_stores.chroma import ChromaDocumentStore

from rich import print  # https://rich.readthedocs.io/en/stable/markup.html#console-markup
from rich.console import Console
from rich.highlighter import RegexHighlighter
from rich.style import Style
from rich.table import Table
from rich.theme import Theme

In [3]:
hl_list = [
    r"(?i)(?P<bankother>phone|bank shot|bank of the|banked)",
    r"(?i)(?P<bank>bank|boa|teller|deposit|vault|account)",

    r"(?i)(?P<boardother>back board|board of direct|board the)",
    r"(?i)(?P<board>board|plank|timber|2x4|wood supplies)",

    r"(?i)(?P<flying>fly over|pilot|navigation|boeing|747|landing)",
    r"(?i)(?P<cool>fly sister|cool)",
    r"(?i)(?P<fly>fly)",
]

class MyHighlighter(RegexHighlighter):
    """Apply style to anything that looks like an email."""
    base_style = "example."
    highlights = hl_list

my_hl = MyHighlighter()

theme = Theme(
    {"example.bankother": Style(color='green', bgcolor="grey93"),
     "example.bank": Style(color='dark_green', bold=True),

     "example.boardother": Style(color='blue', bgcolor="grey93"),
     "example.board": "bold dark_blue",

     "example.cool": Style(color='magenta', bgcolor="grey93"),
     "example.flyother": Style(color='red', bgcolor="grey93"),
     "example.flying": Style(color='dark_red', bold=True),
     })

console = Console(highlighter=my_hl, theme=theme)

In [4]:
print(f"Torchy?? [red]{torch.cuda.is_available()}[/red]")
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

Torchy?? True

In [5]:
from os.path import exists

document_store = ChromaDocumentStore()
# document_store = Pinecone... TODO...
fpath = '../data/test-paragraphs.txt'
file_exists = exists(fpath)
print(f"File exists?? {file_exists}")

File exists?? True

In [6]:
def process_text(content, embedding_mode, document_store):
    print(f"Read content (len:{len(content)}) from path: {fpath}")

    cleaner = DocumentCleaner(remove_repeated_substrings=True)
    big_doc = Document('doc1', content=content)
    print(f"Doc store STARTS with: {document_store.count_documents()} documents")
    splitter = DocumentSplitter(split_by="sentence", split_length=1, split_overlap=0)
    chunked_docs = splitter.run(documents=[big_doc])
    # todo - some better way to do this??
    cnt = 0
    for chunk in chunked_docs["documents"]:
        cnt += 1
        chunk.id = f"{big_doc.id}--{cnt}"

    document_embedder = SentenceTransformersDocumentEmbedder(model=embedding_model)
    document_writer = DocumentWriter(document_store)
    indexing = Pipeline()
    # indexing.add_component(instance=cleaner, name="document_cleaner")
    # indexing.add_component(instance=splitter, name="document_splitter")
    indexing.add_component(instance=document_embedder, name="document_embedder")
    indexing.add_component(instance=document_writer, name="document_writer")
    # indexing.connect("converter", "document_cleaner")
    # indexing.connect("document_cleaner", "document_embedder")
    # indexing.connect("document_splitter", "document_embedder")
    indexing.connect("document_embedder", "document_writer")
    print(f"[green]Indexing {len(chunked_docs['documents'])} documents")
    # print(f"[green]Indexing {big_doc}")
    indexing.run({"document_embedder": {"documents": chunked_docs["documents"]}})
    # indexing.run({"document_cleaner": {"documents": chunked_docs["documents"]}})
    print(f"[green]Doc store AFTER indexing: {document_store.count_documents()} documents")

In [7]:
def do_query(q, embedding_model, k_results, document_store):
    retriever = ChromaQueryTextRetriever(document_store=document_store, top_k=k_results)
    foo = retriever.run(q)

    text_embedder = SentenceTransformersTextEmbedder(model=embedding_model)
    text_embedder.warm_up()
    bar = text_embedder.run(q)

    query_pipeline = Pipeline()
    # query_pipeline.add_component("text_embedder", text_embedder)
    query_pipeline.add_component("retriever", retriever)
    # query_embedding = text_embedder.run(text=query)
    # result = query_pipeline.run({"text_embedder": {"text": query}})
    # result = query_pipeline.run({"retriever":query_embedding})
    results = query_pipeline.run({"retriever": {"query": q, "top_k": k_results}})
    return results

In [8]:
def display_results(table_title, results, show_lines=True, width=150):
    table = Table(title=table_title, show_lines=show_lines, width=width, highlight=True)
    table.add_column("id", style="blue", )  # no_wrap=True
    table.add_column("text")  # no_wrap=True
    table.add_column("Score", style="blue", )
    for d in results["retriever"]["documents"]:
        # print(f"Result: ", d.meta, d.score)
        table.add_row(d.id, d.content.strip(), "{:.2f}".format(d.score))
    console.print(table)
    print('\n\n')

In [9]:
queries = [
    'who is in the bank to deposit money?',
    'who is on a bank?',

    'where can I get a board for fixing my wall?',
    'who is on the board?',

    'who is flying in a plane?',
    'who is looking super fly?',
    'who is looking cool?',
    ]

k = 5

In [13]:
import requests
fpath = 'https://raw.githubusercontent.com/seanoc5/wooly-tongue/main/data/test-paragraphs.txt'
rsp = requests.get(fpath)
if(rsp.status_code == 200):
    print(f"Got the text!")
else:
    print("ERROR: could not get text!!!")
    

Got the text!

In [14]:
content = rsp.text
# print(f"Got content (len:{len(content)} from path: {fpath}, now process it (embed and save to doc store)")
process_text(content, embedding_model, document_store)

Read content (len:1677) from path: 
https://raw.githubusercontent.com/seanoc5/wooly-tongue/main/data/test-paragraphs.txt

Doc store STARTS with: 26 documents

Indexing 26 documents

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Add of existing embedding ID: doc1--1
Insert of existing embedding ID: doc1--1
Add of existing embedding ID: doc1--2
Insert of existing embedding ID: doc1--2
Add of existing embedding ID: doc1--3
Insert of existing embedding ID: doc1--3
Add of existing embedding ID: doc1--4
Insert of existing embedding ID: doc1--4
Add of existing embedding ID: doc1--5
Insert of existing embedding ID: doc1--5
Add of existing embedding ID: doc1--6
Insert of existing embedding ID: doc1--6
Add of existing embedding ID: doc1--7
Insert of existing embedding ID: doc1--7
Add of existing embedding ID: doc1--8
Insert of existing embedding ID: doc1--8
Add of existing embedding ID: doc1--9
Insert of existing embedding ID: doc1--9
Add of existing embedding ID: doc1--10
Insert of existing embedding ID: doc1--10
Add of existing embedding ID: doc1--11
Insert of existing embedding ID: doc1--11
Add of existing embedding ID: doc1--12
Insert of existing embedding ID: doc1--12
Add of existing embedding ID: doc1--13
Insert 

Doc store AFTER indexing: 26 documents

In [15]:
# print("Prepare to run queries...")
for q in queries:
   results = do_query(q, embedding_model, k, document_store)
   title = f"Embedding model: [bold blue]{embedding_model}[/] \nQuery: [bold blue]{q}[/]"
   display_results(title, results, show_lines=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                               Embedding model: sentence-transformers/all-MiniLM-L6-v2             
                                                     Query: who is in the bank to deposit money?                   
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ id       ┃ text                                                                                                  
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ doc1--1  │ Bank of America is a well known bank in the US.                                                       
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--3  │ While at the bank office his mind wandered to thinking about what is currently in the vault--only know
│          │ accountants.                                                                                          
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--2  │ A banker friend of mine, Jaime, works at BoA.                                                         
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--5  │ She worked on the phone bank while they ate.                                                          
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--11 │ From his private airplane, Bill banked left to watch the capybara stampeed.                           
└──────────┴───────────────────────────────────────────────────────────────────────────────────────────────────────

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                               Embedding model: sentence-transformers/all-MiniLM-L6-v2             
                                                               Query: who is on a bank?                            
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ id       ┃ text                                                                                                  
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ doc1--1  │ Bank of America is a well known bank in the US.                                                       
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--2  │ A banker friend of mine, Jaime, works at BoA.                                                         
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--3  │ While at the bank office his mind wandered to thinking about what is currently in the vault--only know
│          │ accountants.                                                                                          
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--5  │ She worked on the phone bank while they ate.                                                          
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--11 │ From his private airplane, Bill banked left to watch the capybara stampeed.                           
└──────────┴───────────────────────────────────────────────────────────────────────────────────────────────────────

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

                                               Embedding model: sentence-transformers/all-MiniLM-L6-v2             
                                                           Query: who is flying in a plane?                        
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ id            ┃ text                                                                                             
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ doc1--24      │ The pilot took the plane in for a smooth landing.                                                
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--21      │ However, there seemed to be a random fly buzzing around both of them.                            
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--11      │ From his private airplane, Bill banked left to watch the capybara stampeed.                      
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--25      │ Clouds changed the navigation of boeing 747, redirecting it away from the storm.                 
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--23      │ Oddly, his mind wandered to a high school baseball game where he dropped a fly ball.             
└───────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                               Embedding model: sentence-transformers/all-MiniLM-L6-v2             
                                                           Query: who is looking super fly?                        
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ id            ┃ text                                                                                             
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ doc1--20      │ His fly sister kept browsing social media promoting her style and general coolness.              
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--21      │ However, there seemed to be a random fly buzzing around both of them.                            
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--23      │ Oddly, his mind wandered to a high school baseball game where he dropped a fly ball.             
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--19      │ Frank decided to fly over the forest and look at the trees and their branches.                   
├───────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--22      │ Frank put down his fly fishing gear on the river bank.                                           
└───────────────┴──────────────────────────────────────────────────────────────────────────────────────────────────

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                               Embedding model: sentence-transformers/all-MiniLM-L6-v2             
                                                             Query: who is looking cool?                           
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ id          ┃ text                                                                                               
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ doc1--20    │ His fly sister kept browsing social media promoting her style and general coolness.                
├─────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--26    │                                                                                                    
├─────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--2     │ A banker friend of mine, Jaime, works at BoA.                                                      
├─────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--7     │ While on the call, she wandered over to Jeff's cube where he was sitting.                          
├─────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────
│ doc1--9     │ Then she thought about Bill Murray, and whether he was sitting on the bank of the river fishing.   
└─────────────┴────────────────────────────────────────────────────────────────────────────────────────────────────

In [11]:
print("Done!")

Done!